# APRENDIZAJE SUPERVISADO

## Importación de librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Lectura del dataset

In [ ]:
URL_TRAIN_DATA = "https://www.famaf.unc.edu.ar/~nocampo043/travel_insurance_prediction_train.csv"
URL_TEST_DATA = "https://www.famaf.unc.edu.ar/~nocampo043/travel_insurance_prediction_test.csv"


df_train = pd.read_csv(URL_TRAIN_DATA)
df_test = pd.read_csv(URL_TEST_DATA)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train

In [ ]:
df_test

## Análisis descriptivo

In [ ]:
df_train.describe().round()

### Variable objetivo o target: Travel Insurance

In [ ]:
df_train["TravelInsurance"].value_counts()

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.countplot(data=df_train, x="TravelInsurance")

### Correlación variables

In [ ]:
corr = df_train[["TravelInsurance","AnnualIncome", "Age"]].corr().round(2)
corr = corr[['TravelInsurance']]
corr.loc[:, 'abs_corr'] = np.abs(corr['TravelInsurance'])
corr.sort_values(by='abs_corr', ascending=False)

In [ ]:
plt.figure(figsize=(10,10))
corr = df_train.corr()
sns.heatmap(corr, xticklabels = corr.columns, yticklabels = corr.columns, annot=True, cmap= 'coolwarm')
plt.show()

### Variable explicativa: Age

En primer lugar, observamos la distribución de la variable Age en el df_train.

In [ ]:
sns.countplot(data=df_train, x="Age", hue="TravelInsurance")

In [ ]:
df_train[["TravelInsurance", "Age"]].groupby("TravelInsurance").describe()

Podemos observar que se encuentran fluctuaciones con respecto a la edad.

### Variable explicativa: AnnualIncome

En primer lugar, observamos la distribución de la variable Annual Income en el df_train.

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.boxenplot(data=df_train, x="AnnualIncome")
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
fig = plt.figure(figsize=(8,8))
sns.boxenplot(data=df_train, x="TravelInsurance", y="AnnualIncome")
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
(
    df_train[["TravelInsurance", "AnnualIncome"]]
        .groupby(["TravelInsurance"])
        .describe()
)

Podemos observar que la distribución de la variable Annual Income se ve afectada en gran medida, al condicionarla por las distintas clases de la variable TARGET. Si bien los valores mínimos y máximos son similares, la media y la mediana difieren considerablemente, como así también el rango intercuantil.
Es una varible que se considera importante para explicar el comportamiento de Y.

### Variable explicativa: Employment Type

In [ ]:
df_train["Employment Type"].value_counts()
pd.crosstab(df_train["TravelInsurance"], df_train["GraduateOrNot"])

In [ ]:
fig = plt.figure(figsize=(5, 5))
sns.countplot(data=df_train,
              x="TravelInsurance",
              hue=df_train["Employment Type"])
pd.crosstab(df_train["TravelInsurance"], df_train["Employment Type"])

### Variable explicativa: Graduate Or Not

In [ ]:
df_train["GraduateOrNot"].value_counts()

In [ ]:
fig = plt.figure(figsize=(5, 5))
sns.countplot(data=df_train,
              x="TravelInsurance",
              hue=df_train["GraduateOrNot"])

In [ ]:
df_train

### Variable explicativa: Frequent Flyer

In [ ]:
df_train["FrequentFlyer"].value_counts()

### Variable explicativa: Ever Travelled Abroad

In [ ]:
df_train["EverTravelledAbroad"].value_counts()

### Variable explicativa: Employment Type

In [ ]:
sns.countplot(data=df_train, x="Employment Type")

### Variable explicativa: Family Members

In [ ]:
df_train.columns

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(data=df_train, x="FamilyMembers", hue= 'TravelInsurance', multiple="stack")

### Variable explicativa: Chronic Diseases

In [ ]:
plt.figure(figsize=(10,8))
ax= sns.countplot(data= df_train, x="ChronicDiseases", hue= "TravelInsurance")
legend_labels, _= ax.get_legend_handles_labels()
ax.legend(legend_labels, ['Not buyed', 'Buyed'], #ver de cambiar!
          title_fontsize = 18,
          fontsize = 15,
          bbox_to_anchor=(1,1),
          title='Travel Insurance')

### Variable explicativa: FrequentFlyer

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.countplot(data=df_train, x="TravelInsurance", hue=df_train.FrequentFlyer)

pd.crosstab(df_train["TravelInsurance"], df_train["FrequentFlyer"])

Se puede observar en este gráfico que si no sos viajero frecuente la cantidad
de clientes que contratan un seguro es baja, caso contrario la cantidad de
contratar un seguro es pareja.

### Variable explicativa: EverTravelledAbroad

In [ ]:
fig = plt.figure(figsize=(5, 5))
sns.countplot(data=df_train, x="TravelInsurance", hue=df_train.EverTravelledAbroad)

pd.crosstab(df_train["TravelInsurance"], df_train["EverTravelledAbroad"])

## Encoding variables

## Creación del train y validation
Para el entrenamiento de los modelos no utilizaremos las variable Customer y
Travel Insurance (Target)

In [ ]:
seed = 0

X_train_total = df_train.drop(["Customer","TravelInsurance"], axis=1)
y_train_total = df_train["TravelInsurance"]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_total,
                                                      y_train_total,
                                                      test_size=0.2,
                                                      random_state=seed)
X_test = df_test.drop("Customer", axis=1)

## Modelos propuestos

## Primer modelo de prueba: Regresión Logística
numerical_cols = X_train_total.select_dtypes(
    include=['float64', 'int64']).columns

categorical_cols = X_train_total.select_dtypes(include=['object']).columns

vars_to_scale = ["Age", "AnnualIncome", "FamilyMembers"]

preprocessor = ColumnTransformer(
    [("encoder", OneHotEncoder(), categorical_cols),
     ("scaler", StandardScaler(), vars_to_scale)],
    remainder="passthrough")

pipe_lgr = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(random_state=seed))
])
pipe_lgr.fit(X_train, y_train)

In [ ]:
pipe_lgr["model"].classes_

In [ ]:
pipe_lgr["model"].get_params()

In [ ]:
y_train_pred_lreg = pipe_lgr.predict(X_train)
y_val_pred_lreg = pipe_lgr.predict(X_valid)
y_test_pred_lreg = pipe_lgr.predict(X_test)

In [ ]:
text = "Logistic Regression - Reporte de clasificación del conjunto de train"
print(len(text)*"=")
print(text)
print(len(text)*"=")
print(classification_report(y_train, y_train_pred_lreg))

In [ ]:
text = "Logistic Regression - Reporte de clasificación del conjunto de validation"
print(len(text)*"=")
print(text)
print(len(text)*"=")
print(classification_report(y_valid, y_val_pred_lreg))

## Segundo modelo de prueba: Árbol de decisión

In [ ]:
pipe_dtree = Pipeline([
    ("preprocessor", preprocessor),
    ("model", DecisionTreeClassifier(random_state=seed))
])
pipe_dtree.fit(X_train, y_train)

In [ ]:
y_train_pred_dtree = pipe_dtree.predict(X_train)
y_valid_pred_dtree = pipe_dtree.predict(X_valid)
y_test_pred_dtree = pipe_lgr.predict(X_test)

In [ ]:
train_acc = accuracy_score(y_train, y_train_pred_dtree)
valid_acc = accuracy_score(y_valid, y_valid_pred_dtree)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Val accuracy: {valid_acc:0.2}')